<a href="https://colab.research.google.com/github/AndrewDiv/PMfMIVV/blob/main/DiplomaUpd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [22]:
import numpy as np
import time
from scipy import optimize

# Simulator

In [23]:
def Prevalence(host_num, single_strain_prob = 0.3):
  """ Simulate prevalence of strains in the host
    
  Parameters
  ----------
  host_num : int
    Number of patients
  single_strain_prob : float
    The probability that the host carries only one strain
       
  Returns
  -------
  prevs: ndarray, shape (host_num,)
    List of prevalence of one strain
  """
  prevs = np.zeros(host_num)
  for i in range(host_num):
    rnd = np.random.uniform()
    if rnd < single_strain_prob:
      prevs[i] = np.random.choice([0., 1.])
    else:
      prevs[i] = np.random.uniform()
  return prevs

In [3]:
def pi_i(r_s, prevs, er_rate):
  return prevs*(r_s[0]*(1 - er_rate) + (1 - r_s[0])*er_rate) + (1-prevs)*(r_s[1]*(1 - er_rate) + (1 - r_s[1])*er_rate)

def SimulateReads(haps, prevs, coverage = 100,  er_rate = 0.01):
  """ Simulate reads from one host
    
  Parameters
  ----------
  haps : ndarray, shape (n, m)
    List of haplotypes
  prevs: ndarray, shape (k,)
    List of prevalences
  coverage: int
    Number of reads that cover sites
  er_rate: float
    Error rate 
       
  Returns
  -------
  data: ndarray, shape (host_num, hap_len, hap_num)
    Array of the number of observations of different alleles at each site
     for all patients
  """
  hap_len = len(haps[0])
  hap_num = len(haps)
  host_num = prevs.shape[0]
  data = np.empty((host_num, hap_len, hap_num))
  #print(hap_len, hap_num)
  for host in range(host_num):
    for snv in range(hap_len):
      als = np.zeros(hap_num)
      lcoverage = coverage
      als[1] = np.random.binomial(lcoverage, pi_i([haps[0][snv], haps[1][snv]], prevs[host], er_rate))
      als[0] = lcoverage - als[1]
      data[host][snv] = als
    
  return data

# Log-Likelihood

In [4]:
def LogFactorial(n):
  return sum(np.log(np.arange(1, n+1)))

def Coeff(reads):
  coeff = 0
  for _reads in reads:
    coeff += LogFactorial(_reads[0] + _reads[1]) - LogFactorial(_reads[1]) - LogFactorial(_reads[0])
  return coeff

In [50]:
def SingleHostLLH(prev, haps, samp_reads, er_rate=0.):
  """ Calculate minus log-likelihood of a single host
    
  Parameters
  ----------
  prev : float
    prevalence of 1st strain
  haps : ndarray, shape (hap_num, m)
    List of haplotypes
  samp_reads: ndarray, shape (m, hap_num)
    reads of host
  er_rate: float
    Error rate 
       
  Returns
  -------
  -llh: float
    log-likelihood of a single host
  """
  llh = Coeff(samp_reads)
  for snv in range(len(samp_reads)):
    p = prev*(haps[0][snv]*(1.0 - er_rate) + (1.0 - haps[0][snv])*er_rate) + (1-prev)*(haps[1][snv]*(1.0 - er_rate) + (1.0 - haps[1][snv])*er_rate)
    if p <= 0:
      print([haps[i][snv] for i in range(len(prev))])
      print("error rate: ", er_rate) 
      print("prevs:", prev)
    
    llh += samp_reads[snv][1] * np.log(p) + samp_reads[snv][0] * np.log(1.0 - p)
  return -llh

def LLH(haps, prevs, samples, num_of_haplotypes, er_rate):
  """ Calculate minus log-likelihood of the hosts
    
  Parameters
  ----------
  haps : ndarray, shape (hap_num, m)
    List of haplotypes
  prevs: ndarray, shape (k,)
    List of prevalences of 1st strain
  samples: ndarray, shape (n, m, hap_num)
    reads of all hosts
  num_of_haplotypes: int
    hap_num
  er_rate: float
    Error rate 
       
  Returns
  -------
  -llh: float
    log-likelihood of the whole sample
  """  
  haps = np.reshape(haps, (num_of_haplotypes, -1))
  #print(haps)
  llh = 0
  for sample_num in range(len(samples)):
      llh += SingleHostLLH(prevs[sample_num], haps, samples[sample_num], er_rate)
  #print('LLH:', llh)
  return llh

In [43]:
def optimise_prevs_const_haps(haps, prevs, samp_reads, hap_num=2, er_rate=0.):
  haps = np.reshape(haps, (hap_num, -1))
  #prevs = np.reshape(prevs, (len(samp_reads), -1))
  #print(prevs.shape)
  llh = 0.
  opt_prevs = np.empty(len(prevs))
  bnds = ((0, 1),)
  for host_num in range(len(samp_reads)):
    opt_res = optimize.minimize(SingleHostLLH, prevs[host_num], args=(haps, samp_reads[host_num], er_rate),
                                method='trust-constr', bounds=bnds, options = {'xtol': 1e-6, 'gtol': 1e-6, 'barrier_tol': 1e-6, 'maxiter': 100})
    opt_prevs[host_num] = opt_res.x
    llh += opt_res.fun
    
  return opt_prevs

In [42]:
def optimise_haps_const_prevs(haps, prevs, samp_reads, num_of_hapl=2, er_rate=0.):
  #print(haps)
  prevs = np.reshape(prevs, (len(samp_reads), -1))
  bnds = ((0, 1), ) * len(haps)
  opt_res = optimize.minimize(LLH, haps, args=(prevs, samp_reads, num_of_hapl, er_rate),
                                method='trust-constr', bounds=bnds, 
                                options = {'xtol': 1e-6, 'gtol': 1e-6, 'barrier_tol': 1e-6, 'maxiter': 100})
  opt_haps = opt_res.x

  return opt_haps

# Testing

In [9]:
exmp_haplotypes = [[1., 1., 0., 0.],
              [0., 0., 1., 1.]] # Probabilistic description of haplotypes

hap_len = len(exmp_haplotypes[0]) # Number of sites
coverage = 100 # coverage sites
hap_num = len(exmp_haplotypes)  # Number of haplotypes(strains)
samp_num = 10  # Number of samples(patients)
er_rate = 0.01  # Error rate

In [20]:
exmp_prevs = Prevalence(samp_num, 0.3)
print(exmp_prevs)

patients_data = SimulateReads(exmp_haplotypes, exmp_prevs, er_rate= 0.)
print(patients_data)



[0.35571709 0.         0.29360537 0.66695958 0.79481425 0.
 0.57836888 0.14282305 0.36709016 0.06722632]
<class 'numpy.ndarray'>


In [ ]:
np.random.seed(2022)
start_time = time.time()

haps_init = np.random.uniform(size = hap_num * hap_len)
prev_init = np.random.uniform(size = samp_num)


opt_haps = optimise_haps_const_prevs(haps_init, prev_init, patients_data)
opt_prevs = optimise_prevs_const_haps(opt_haps, prev_init, patients_data)

for _ in range(6):
    opt_haps = optimise_haps_const_prevs(opt_haps, opt_prevs, patients_data)
    opt_prevs = optimise_prevs_const_haps(opt_haps, opt_prevs, patients_data)

print("time elapsed: {:.2f}s".format(time.time() - start_time))

print('RESULT')
print("opt_haps:", np.reshape(opt_haps, (hap_num, -1)))
print("real_haps:", exmp_haplotypes)
print("opt_prevs:", opt_prevs)
print("real_prevs:", exmp_prevs)

#Simultaneous optimisation